In [624]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from clean_data_testing import *
from utils import split_data, stratified_train_test_split
import datetime
import seaborn as sns
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [622]:
from functions import *

In [623]:
from clean_data_testing import *

In [625]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [626]:
x_tr, x_te, y_tr, y_te = stratified_train_test_split(x_train, y_train, test_size=0.2, random_seed= 42)

In [739]:
x_train_cleaned, features,median_and_most_probable_class  = clean_data_x(x_tr, labels)

    
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/x_trained.cleaned.csv", x_train_cleaned, delimiter=",")

In [740]:
x_te_cleaned = clean_data_testing(x_te, labels, features, median_and_most_probable_class)

In [741]:
print(x_train_cleaned.shape)

(262508, 20)


In [742]:
def slice_data(x_train, y_train, num_slices):
    # Determine the size of each slice for both training and test sets
     # Shuffle indices for training data
    np.random.seed(42)
    train_indices = np.random.permutation(len(x_train))

    x_train_shuffled = x_train[train_indices]
    y_train_shuffled = y_train[train_indices]
    train_slice_size = len(x_train) // num_slices
    x_train_slices = [x_train_shuffled[i * train_slice_size: (i + 1) * train_slice_size] for i in range(num_slices)]
    y_train_slices = [y_train_shuffled[i * train_slice_size: (i + 1) * train_slice_size] for i in range(num_slices)]
    
    
    return x_train_slices, y_train_slices

In [743]:
num_samples = x_train_cleaned.shape[0]
number_of_votes = 1
tx = np.c_[np.ones(num_samples), x_train_cleaned]
x_train_slices, y_train_slices = slice_data(tx, y_train, number_of_votes)

In [800]:
max_iters = 1000
gamma = 0.1
batch_size = 64
lambda_ =  0.05
alpha = 1
num_samples = x_train_cleaned.shape[0]



# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution


w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])
w_list = []
for i in range(number_of_votes) :
    sgd_losses, sgd_ws = stochastic_gradient_descent(
    y_train_slices[i], x_train_slices[i], w_initial, batch_size, max_iters, gamma, lambda_, alpha
    )
    w_list.append(sgd_ws[-1])

In [801]:
def voting_system(model_list, tx_te) :
    mean_predictions = 0
    for model in model_list :
        mean_predictions += sigmoid(tx_te@model)
    y_predict = np.where(mean_predictions/len(model_list) >= 0.5, 1, -1)
    return y_predict 

In [802]:
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = voting_system(w_list, tx_te)

In [803]:
accuracy(y_te, y_predict)

0.9116674539442607

In [804]:
f1_score(y_te, y_predict)

(0.0003448870494912916, 0.0001725625539257981)

In [769]:
np.sum(y_predict  == 1)

19732